In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from collections import Counter

mails = pd.read_csv("spam.csv",encoding='latin-1',header=0,names=["labels","message"],usecols=["labels","message"])
mails["labels"]=mails["labels"].replace("ham",0)
mails["labels"]=mails["labels"].replace("spam",1)

X = mails["message"]
Y = mails["labels"]

In [2]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,random_state=0,test_size=.3)
print(y_test.size)

1672


In [3]:
#vectorizer = CountVectorizer()
#counts = vectorizer.fit_transform(X.values)

#target = y_train.values
#print(counts)

In [4]:
c = Counter(Y)
numHam = c[0]
numSpam = c[1]

In [7]:
SpamWord = {}
HamWord = {}
SpamWordTotal = 0
HamWordTotal = 0
pSpam = 0
pHam = 0

def train():
    for i in x_train.index:
        email = x_train[i]
        label = y_train[i]
        processEmail(email,label)
    pSpam = math.log(numSpam/Y.size)
    pHam = math.log(numHam/Y.size)
    
def processEmail(message, labels):
    message=stopWords(message)
    for word in message:
        if labels == 1:
            SpamWord[word] = SpamWord.get(word, 0) + 1
            global SpamWordTotal
            SpamWordTotal += 1
        else:
            HamWord[word] = HamWord.get(word, 0) + 1
            global HamWordTotal
            HamWordTotal += 1

def stopWords(message):
    newmessage=''
    stop=['about','and','are','com','for','from','how','that','the','this', 'was','what','when','where','who','will','with','und','the','www']
    for word in message:
        if word != stop:
            newmessage = newmessage + word
    return newmessage

def conditionalEmail(message , spam):
    result = 1.0
    message=stopWords(message)
    for word in message:
        result *= conditionalWord(message, spam)
    return result

def classify(email):
    isSpam = pSpam + conditionalEmail(email, True) 
    isHam = pHam + conditionalEmail(email, False) 
    return isSpam > isHam

def conditionalWord(word, spam):
    if spam:
        return math.log(SpamWord.get(word,0)+1)/(float)(SpamWordTotal + (1*(SpamWordTotal + HamWordTotal)))
    return math.log(HamWord.get(word,0)+1)/(float)(HamWordTotal + (1*(SpamWordTotal + HamWordTotal)))



train()


correct = 0
for i in x_test.index:
    if (classify(x_test[i]) == True) and (y_test[i] == 1):
        correct +=1
    if (classify(x_test[i]) == False) and (y_test[i] == 0):
        correct +=1
        
Accuracy =(correct/y_test.size) * 100
print("Accuracy = ", Accuracy)

Accuracy =  85.76555023923444
